In [ ]:
#导入包

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import backend, models, layers, optimizers, regularizers
from tensorflow.keras.utils import to_categorical
import numpy as np
from numpy import expand_dims 
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from IPython.display import display 
from PIL import Image 
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.preprocessing.image import array_to_img
from keras.applications.vgg19 import preprocess_input
from keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers,regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Input
from tensorflow.keras import optimizers,regularizers
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import os, shutil, datetime
from keras.models import Sequential, Model
from keras.layers import Input 
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense 
from keras.layers import Flatten 
from keras.layers import Dropout 
from keras.optimizers import SGD, Adam
import pandas as pd
from IPython.core.display import display, HTML

import os
from statistics import mean, median
from numpy.random import seed

from statistics import mean, median
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import math
from sklearn.metrics import classification_report
from keras.models import load_model
from tensorflow.keras.utils import  plot_model     

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#参数设置

seed(1234)
base       = '../input/birdafter-1/'
train_dir  = 'train_1/'
test_dir   = 'test_1/'
valid_dir  = 'val_1/'
DIR_TEST =  base + test_dir
DIR_TRAIN = base + train_dir
DIR_VALID = base + valid_dir

b_epochs = 30
b_size = 32
b_lr=0.0001

img_width =  224
img_height = 224


callbacks_list = [
  EarlyStopping(
        monitor = 'accuracy', 
        patience = 8, 
        mode = 'max',
        restore_best_weights=True),
  ReduceLROnPlateau( 
      monitor='accuracy',
      factor=0.1, 
      patience=1)
]

In [ ]:
# 数据统计
def cntSamples(directory):
    specs = []
    for root, dirs, files in os.walk(directory, topdown=True):
        dirs.sort()
        for name in dirs:
            if name not in specs:
                specs.append(name)

    # file counts for each species 
    nums = []
    for b in specs:
        path = os.path.join(directory,b)
        num_files = len(os.listdir(path))
        nums.append(num_files)
 
    # Create Dictionary
    adict = {specs[i]:nums[i] for i in range(len(specs))}
    return adict
testDict =  cntSamples(DIR_TEST)
trainDict = cntSamples(DIR_TRAIN)
validDict = cntSamples(DIR_VALID)
num_classes = len(trainDict)


# Count labels in train, valid and test


train_tbl = pd.DataFrame.from_dict(trainDict, 
                orient='index', dtype=None, columns=['Images'])

num_classes = len(trainDict)
label_index = list(range(num_classes))
#print(num_classes)
train_tbl.insert(0,'Label Index',label_index, True)

# Creates pandas DataFrame. 
data = {'样本数': [sum(trainDict.values()), 
                   sum(testDict.values()), 
                   sum(validDict.values())],
        
        '样本种类': [len(trainDict.values()), 
                    len(testDict.values()), 
                    len(validDict.values())],
        
        '最小样本量': [min(trainDict.values()), 
                    min(testDict.values()), 
                    min(validDict.values())],
        
        '最大样本量': [max(trainDict.values()), 
                    max(testDict.values()), 
                    max(validDict.values())],
        
        '平均样本量': [int(median(trainDict.values())), 
                   int(median(testDict.values())), 
                   int(median(validDict.values()))]}
lblSummary = pd.DataFrame(data, index =['Train', 'Test', 'Valid']) 
display(HTML(lblSummary.to_html()))

num_train=sum(trainDict.values())
num_val=sum(testDict.values())

'''
def tuple_count(file_path, dataset):
    bird_count = []
    for file in os.listdir(file_path):
        bird_count.append((file, len(os.listdir(file_path + file)), dataset))
    return bird_count

consolidated = tuple_count(DIR_TEST, '测试集样本量') + tuple_count(DIR_VALID, '验证集样本量') + tuple_count(DIR_TRAIN, '训练集样本量')
count_df = pd.DataFrame.from_records(consolidated, columns =['种类名称', '样本量', '数据集划分']) 

fig = px.bar(count_df, x='种类名称', y='样本量', color='数据集划分')
fig.update_xaxes(visible=False)
fig.show()
'''

In [ ]:
'''
# 查看图像
def getKeybyValue(LabelDict, value):
    listItems = LabelDict.items()
    for item in listItems:
        if item[1] == value:
            return item[0]
    
    return None

seed(1234)

def pltImages(dir):
    datagen2 = ImageDataGenerator()
    it2 = datagen2.flow_from_directory(
            dir,
            target_size=(224, 224),
            batch_size=4,
            class_mode='binary')

    labDict = it2.class_indices
    batchX, batchy = it2.next() 
    num_img = batchX.shape[0]
    imgs = [array_to_img(batchX[i]) for i in range(num_img)]
    indx = [int(batchy[i]) for i in range(len(batchy))]
    labs = [getKeybyValue(labDict, i) for i in indx]
   
    # settings
    h, w = 5, 5        
    nrows, ncols = 1, 4  
    figsize = [25,4]     

    # create figure (fig), and array of axes (ax)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi = 80)

    # plot image on each sub-plot
    for i, axi in enumerate(ax.flat):
        # i runs from 0 to (nrows*ncols-1)
        # axi is equivalent with ax[rowid][colid]
        axi.imshow(imgs[i], aspect = 'auto')

        # write Label as title
        axi.set_title(labs[i])

    plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.2, wspace=0.2)
    plt.show()
    return

for d in [train_dir,test_dir,valid_dir]:
    print('\n\n样本示例： ', d)
    pltImages(base + d)
'''

In [ ]:
'''# 搭建InceptionV3模型

if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width, img_height)
else:
    input_shape = (img_width, img_height,3)
    
    
b_epochs = 30
b_size = 32
b_lr=0.0001


# create a data generator 

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 

backend.clear_session()
input_shape = (img_width, img_height, 3)

from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

# Input Tensor
input_tensor = Input(shape=input_shape)  

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

flat_1 = Flatten()(base_model.output)
x = Dense(512, activation='relu')(flat_1)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)
print("layers in model: ", len(model.layers))

model.compile(optimizer=Adam(lr=b_lr), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()
# Train
history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            verbose=1)

end = datetime.datetime.now()
elapsed = end - start
print ('Inception模型训练完成. 用时: ', elapsed)
#model.save("./inception_e50_b32_lr0.0015_sgd_nodrop.h5")
import pickle
with open('./51_inception_e30_b32_lr0001_adam_nodrop_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
'''

In [ ]:
'''
from tensorflow.keras.layers import Input
from tensorflow import keras
import seaborn as sns
from tensorflow.compat.v1 import ConfigProto
from tensorflow.keras.models import Model
config = ConfigProto()
config.gpu_options.allow_growth = True
from keras.preprocessing import image
# 搭建densenet模型

if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width, img_height)
else:
    input_shape = (img_width, img_height,3)


b_epochs = 30
b_size = 32
b_lr=0.0001


    
# create a data generator 

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 

backend.clear_session()
input_shape = (img_width, img_height, 3)



# Input Tensor
input_tensor = Input(shape=input_shape)  

base_model_2 = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3))
def densenet(input_shape=(224, 224, 3)):
    inputs=Input(shape=input_shape)
    x=base_model_2(inputs)
    x=tf.keras.layers.Conv2D(256,4,2,padding='same',activation='relu')(x)
    x=tf.keras.layers.Conv2D(512, 4, 2, padding='same', activation='relu')(x)
    #x=tf.keras.layers.Dropout(0.35)(x)
    s = Model(inputs=base_model_2.input, outputs=base_model_2.get_layer('conv2_block6_2_conv').output)
    s1 = Model(inputs=base_model_2.input, outputs=base_model_2.get_layer('conv3_block2_1_conv').output)
    s=s(inputs)
    s1=s1(inputs)
    x=tf.keras.layers.Conv2D(256,4,2,padding='same',activation='relu')(x)
    x1=tf.keras.layers.GlobalAveragePooling2D()(x)
    x2 = tf.keras.layers.GlobalAveragePooling2D()(s)
    x3 = tf.keras.layers.GlobalAveragePooling2D()(s1)
    x=tf.keras.layers.Concatenate()([x1,x2,x3])
    output_layer=tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return Model(inputs=inputs, outputs=output_layer)
model=densenet((224, 224, 3))
model.summary()

#tf.keras.utils.plot_model(model, to_file='model4.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0015, momentum=0.8), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()
# Train
history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            verbose=1)

end = datetime.datetime.now()
elapsed = end - start
print ('Densenet模型训练完成. 用时: ', elapsed)
#model.save("./51_densenet_e50_b32_lr0001_adam_drop.h5")
import pickle
with open('./51_densenet201_e30_b32_lr0015_sgd_nodrop_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
        
'''

In [ ]:
# 搭建densenet模型
from tensorflow.keras import optimizers,regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width, img_height)
else:
    input_shape = (img_width, img_height,3)


b_epochs = 30
b_size = 32
b_lr=0.00005


    
# create a data generator 

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 


input_shape = (img_width, img_height, 3)



# Input Tensor
input_tensor = Input(shape=input_shape)  

base_model_2 = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3))
def densenet(input_shape=(224, 224, 3)):
    inputs=Input(shape=input_shape)
    x=base_model_2(inputs)
    x=tf.keras.layers.Conv2D(256,4,2,padding='same',activation='relu')(x)
    x=tf.keras.layers.Conv2D(512, 4, 2, padding='same', activation='relu')(x)
    x=tf.keras.layers.Dropout(0.35)(x)
    s = Model(inputs=base_model_2.input, outputs=base_model_2.get_layer('conv2_block6_2_conv').output)
    s1 = Model(inputs=base_model_2.input, outputs=base_model_2.get_layer('conv3_block2_1_conv').output)
    s=s(inputs)
    s1=s1(inputs)
    x=tf.keras.layers.Conv2D(256,4,2,padding='same',activation='relu')(x)
    x1=tf.keras.layers.GlobalAveragePooling2D()(x)
    x2 = tf.keras.layers.GlobalAveragePooling2D()(s)
    x3 = tf.keras.layers.GlobalAveragePooling2D()(s1)
    x=tf.keras.layers.Concatenate()([x1,x2,x3])
    output_layer=tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return Model(inputs=inputs, outputs=output_layer)
model=densenet((224, 224, 3))


tf.keras.utils.plot_model(model, to_file='model4.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)
model.compile(optimizer=optimizers.Adam(lr=b_lr), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()
# Train
history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            verbose=1)

end = datetime.datetime.now()
elapsed = end - start
print ('Densenet模型训练完成. 用时: ', elapsed)
#model.save("./densenet_e50_b32_lr0.0001_adam_drop.h5")
import pickle
with open('./6_densenet201_e30_b32_lr00005_adam_35drop_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
        

In [ ]:
'''
from keras.applications import ResNet50V2
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
# 搭建resnet50模型

b_epochs = 30
b_size = 32
b_lr=0.0001


if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width, img_height)
else:
    input_shape = (img_width, img_height,3)
    
# create a data generator 

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 

backend.clear_session()


base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in base_model.layers:
    layer.trainable=False
plot_model(base_model)

model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax',kernel_initializer='glorot_uniform'))
print(model.summary())

#model.summary()
#tf.keras.utils.plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)
model.compile(optimizer=optimizers.Adam(lr=b_lr), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()
# Train
history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            verbose=1)

end = datetime.datetime.now()
elapsed = end - start
print ('/Resnet50模型训练完成. 用时: ', elapsed)
model.save("./resnet50_e50_b8_lr0001_adam_drop.h5")
import pickle
with open('./resnet50_e50_b8_lr0001_adam_drop_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
'''

In [ ]:
'''
#vgg16
#import libraries
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input


b_epochs = 30
b_size = 32
b_lr=0.0001


# create a data generator 

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 

backend.clear_session()

#加载迁移学习的预训练权重
base_model=keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))
#方法一：冻结预测层之前的所有的权重参数
base_model.trainable = False

#Create new model on top
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(200,activation='softmax',kernel_initializer='glorot_normal'))
model.summary()
#tf.keras.utils.plot_model(model, to_file='vgg16model.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)
print("layers in model: ", len(model.layers))

model.compile(optimizer=Adam(lr=b_lr), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()


history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            verbose=1)


end = datetime.datetime.now()
elapsed = end - start
print ('/Vgg16模型训练完成. 用时: ', elapsed)
#model.save("./51_vgg16v1_e30_b32_lr0001_adam_nodrop.h5")
import pickle
with open('./2_vgg16v1_e30_b32_lr0001_adam_50drop_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
'''

In [ ]:
'''
#mydensenet
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dense,Flatten,Dropout,MaxPooling2D,Input,Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import  matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

b_size=32
b_epochs=30
b_lr=0.00015

class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, growth_rate, drop_rate):
        super(BottleNeck, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=4 * growth_rate,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=growth_rate,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.dropout = tf.keras.layers.Dropout(rate=drop_rate)

    def call(self, inputs, training=None, **kwargs):
        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.dropout(x, training=training)
        return x


class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate, drop_rate):
        super(DenseBlock, self).__init__()
        self.num_layers = num_layers
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate
        self.features_list = []
        self.bottle_necks = []
        for i in range(self.num_layers):
            self.bottle_necks.append(BottleNeck(growth_rate=self.growth_rate, drop_rate=self.drop_rate))

    def call(self, inputs, training=None, **kwargs):
        self.features_list.append(inputs)
        x = inputs
        for i in range(self.num_layers):
            y = self.bottle_necks[i](x, training=training)
            self.features_list.append(y)
            x = tf.concat(self.features_list, axis=-1)
        self.features_list.clear()
        return x


class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filters=out_channels,
                                           kernel_size=(1, 1),
                                           strides=1,
                                           padding="same")
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                              strides=2,
                                              padding="same")

    def call(self, inputs, training=None, **kwargs):
        x = self.bn(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv(x)
        x = self.pool(x)
        return x


class DenseNet(tf.keras.Model):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, drop_rate):
        super(DenseNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters=num_init_features,
                                           kernel_size=(7, 7),
                                           strides=2,
                                           padding="same")
        self.bn = tf.keras.layers.BatchNormalization()
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                              strides=2,
                                              padding="same")
        self.num_channels = num_init_features
        self.dense_block_1 = DenseBlock(num_layers=block_layers[0], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[0]
        self.num_channels = compression_rate * self.num_channels
        self.transition_1 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_2 = DenseBlock(num_layers=block_layers[1], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[1]
        self.num_channels = compression_rate * self.num_channels
        self.transition_2 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_3 = DenseBlock(num_layers=block_layers[2], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[2]
        self.num_channels = compression_rate * self.num_channels
        self.transition_3 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_4 = DenseBlock(num_layers=block_layers[3], growth_rate=growth_rate, drop_rate=drop_rate)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=200,activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv(inputs)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool(x)

        x = self.dense_block_1(x, training=training)
        x = self.transition_1(x, training=training)
        x = self.dense_block_2(x, training=training)
        x = self.transition_2(x, training=training)
        x = self.dense_block_3(x, training=training)
        x = self.transition_3(x, training=training)
        x = self.dense_block_4(x, training=training)

        x = self.avgpool(x)
        x = self.fc(x)

        return x


def densenet_121():
    return DenseNet(num_init_features=64, growth_rate=32, block_layers=[1, 1, 1, 5], compression_rate=0.35, drop_rate=0.2)


def densenet_169():
    return DenseNet(num_init_features=64, growth_rate=32, block_layers=[6, 12, 32, 32], compression_rate=0.35, drop_rate=0.2)


def densenet_201():
    return DenseNet(num_init_features=64, growth_rate=32, block_layers=[6, 12, 48, 32], compression_rate=0.35, drop_rate=0.2)


#model_1=densenet_121()
model_1=densenet_169()
tf.keras.utils.plot_model(model_1, to_file='model.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)
model = tf.keras.Sequential([model_1])
from tensorflow.keras.utils import  plot_model 
model.build(input_shape=(None, 224, 224, 3))

datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Train Generator for InceptionV3
train_generator = datagen.flow_from_directory(
    base + train_dir , 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Validation Generator 
validation_generator = datagen.flow_from_directory(
    base + valid_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical', 
    batch_size=b_size) 

# Test Generator 
test_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 
model.summary()

model.compile(optimizer=optimizers.Adam(lr=b_lr), 
              loss='categorical_crossentropy', metrics = ['accuracy'])
start = datetime.datetime.now()
# Train
#callback=[tf.keras.callbacks.ModelCheckpoint(filepath='./model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',save_weights_only=True,monitor='val_loss',verbose=0)]
history = model.fit_generator(train_generator, 
                            steps_per_epoch=num_train//b_size, 
                            validation_data=validation_generator, 
                            validation_steps=num_val//b_size, 
                            epochs=b_epochs, 
                            #callbacks = callback,
                            verbose=1)

end = datetime.datetime.now()
elapsed = end - start
print ('/mydensenet模型训练完成. 用时: ', elapsed)
import pickle
with open('./51_mydensenet169_e30_b32_lr00015_adam_hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
'''

In [ ]:
def plot_eval(Model, hist):
    import matplotlib.pyplot as plt
    
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(1, len(loss) + 1)
    
    f = plt.figure(figsize=(12,6))
    ax1 = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    
    plt.clf()   # clear figure
    plt.subplot(121)
    plt.plot(epochs, loss, color = 'blue', label='loss')
    plt.plot(epochs, val_loss, color = 'orange', label='val loss')
    plt.title('Loss')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    plt.subplot(122)
    plt.plot(epochs, acc, color = 'blue', label='acc')
    plt.plot(epochs, val_acc, color = 'orange', label='val acc')
    plt.title('Acc')
    plt.xlabel('epochs')
    plt.ylabel('acc')
    plt.legend()
    plt.show()
    
    acc = hist['accuracy']
    xs = list(range(len(acc)))
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xs, y=acc, mode='lines+markers', name='训练集准确值'))
    fig.add_trace(go.Scatter(x=xs, y=val_acc, mode='lines+markers', name='验证集准确值'))
    fig.add_trace(go.Scatter(x=xs, y=loss, mode='lines+markers', name='训练集损失值'))
    fig.add_trace(go.Scatter(x=xs, y=val_loss, mode='lines+markers', name='验证集损失值'))
    fig.show()
    
loss, accuracy = model.evaluate_generator(test_generator) 
 
# Report
hist = pd.DataFrame(history.history)
e_exe = hist.shape[0]

print("\n\n模型训练与测试\n\n")
print("  迭代轮数: ", e_exe)
print("  初始学习率: ", b_lr)
print("  样本批次量: ", b_size)
print("  损失值: {0:.4f}".format(loss))
print("  准确率: {0:.4f} % ".format(accuracy * 100.0))
print("  训练用时:",elapsed)
plot_eval("Model", hist)

In [ ]:
'''
#测试集报告
pred_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 

pred = model.predict_generator(pred_generator, steps= math.ceil(num_val/b_size))
pred_generator.reset
species = list(train_tbl.index)

pred_labs = np.argmax(pred,axis=1)

#print("测试集预测结果表：\n",pred_labs)
#test_generator.reset()
pred_generator.reset
repDict = classification_report(pred_generator.labels, pred_labs, target_names=species, output_dict = True)
report = classification_report(pred_generator.labels, pred_labs, target_names=species)
print("测试集预测结果分析：\n",report)

pred_generator.reset
errors = np.where(pred_labs != pred_generator.classes)[0]
print("预测错误的图像有%d张"%int(len(errors)))
'''

In [ ]:
'''
import pickle
import matplotlib.pyplot as plt
with open('../input/inception-sgd-adam-hist/inception_e50_b32_lr0001_adam_nodrop_hist', 'rb') as f:
    hist_sgd = pickle.load(f)
    
with open('../input/inception-sgd-adam-hist/inception_e50_b32_lr0015_sgd_nodrop_hist', 'rb') as f:
    hist_adam = pickle.load(f)

plt.figure(figsize=(10,8))
plt.plot(hist_sgd["loss"],linewidth=3.0,color="blue",label="Adam Loss")
plt.plot(hist_adam["loss"],linewidth=3.0,color="red",label="Sgd Loss")
plt.plot(hist_sgd["val_loss"],linewidth=2.0,color="blue",linestyle='--',label="Adam val_Loss")
plt.plot(hist_adam["val_loss"],linewidth=2.0,color="red",linestyle='--',label="Sgd val_Loss")
plt.legend()
plt.title("Loss Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss Values")
plt.grid()
plt.savefig("3.png")
plt.show()


plt.figure(figsize=(10,8))
plt.plot(hist_sgd["accuracy"],color="blue",linewidth=3.0,label="Adam accuracy")
plt.plot(hist_adam["accuracy"],color="red",linewidth=3.0,label="Sgd accuracy")
plt.plot(hist_sgd["val_accuracy"],linewidth=2.0,color="blue",linestyle='--',label="Adam val_accuracy")
plt.plot(hist_adam["val_accuracy"],linewidth=2.0,color="red",linestyle='--',label="Sgd val_accuracy")
plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Values")
plt.savefig("4.png")
plt.grid()
plt.show()
''''''

In [ ]:
'''
import pickle
import matplotlib.pyplot as plt
with open('../input/5cnn-hist/ResNet101V2_e30_b32_lr0001_adam_drop_hist', 'rb') as f:
    hist_res101 = pickle.load(f)
    
with open('../input/5cnn-hist/inception_e30_b32_lr0001_adam_nodrop_hist', 'rb') as f:
    hist_ince = pickle.load(f)

with open('../input/5cnn-hist/resnet50_e30_b32_lr0001_adam_drop_hist', 'rb') as f:
    hist_res50 = pickle.load(f)
     
with open('../input/5cnn-hist/vgg16_e30_b32_lr0001_adam_nodrop_hist', 'rb') as f:
    hist_vgg16v1 = pickle.load(f)
    
with open('../input/5cnn-hist/vgg16v2_e30_b32_lr0001_adam_nodrop_hist', 'rb') as f:
    hist_vgg16v2 = pickle.load(f)
    
plt.figure(figsize=(10,8))
plt.plot(hist_vgg16v1["accuracy"],color="black",linewidth=3.0,label="Vgg16v1 accuracy")
plt.plot(hist_vgg16v2["accuracy"],color="blue",linewidth=3.0,label="Vgg16v2 accuracy")
plt.plot(hist_res50["accuracy"],color="red",linewidth=3.0,label="ResNet50 accuracy")
plt.plot(hist_res101["accuracy"],color="orange",linewidth=3.0,label="ResNet101 accuracy")
plt.plot(hist_ince["accuracy"],color="green",linewidth=3.0,label="Inception accuracy")
plt.plot(hist_vgg16v1["val_accuracy"],color="black",linewidth=1.5,linestyle='--',label="Vgg16v1 val_accuracy")
plt.plot(hist_vgg16v2["val_accuracy"],color="blue",linewidth=1.5,linestyle='--',label="Vgg16v2 val_accuracy")
plt.plot(hist_res50["val_accuracy"],color="red",linewidth=1.5,linestyle='--',label="ResNet50 val_accuracy")
plt.plot(hist_res101["val_accuracy"],color="orange",linewidth=1.5,linestyle='--',label="ResNet101 val_accuracy")
plt.plot(hist_ince["val_accuracy"],color="green",linewidth=1.5,linestyle='--',label="Inception val_accuracy")
plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Values")
plt.grid()
plt.savefig("5.png")
plt.show()

plt.figure(figsize=(10,8))
plt.plot(hist_vgg16v1["loss"],color="black",linewidth=3.0,label="Vgg16v1 loss")
plt.plot(hist_vgg16v2["loss"],color="blue",linewidth=3.0,label="Vgg16v2 loss")
plt.plot(hist_res50["loss"],color="red",linewidth=3.0,label="ResNet50 loss")
plt.plot(hist_res101["loss"],color="orange",linewidth=3.0,label="ResNet101 loss")
plt.plot(hist_ince["loss"],color="green",linewidth=3.0,label="Inception loss")
plt.plot(hist_vgg16v1["val_loss"],color="black",linewidth=1.5,linestyle='--',label="Vgg16v1 val_loss")
plt.plot(hist_vgg16v2["val_loss"],color="blue",linewidth=1.5,linestyle='--',label="Vgg16v2 val_loss")
plt.plot(hist_res50["val_loss"],color="red",linewidth=1.5,linestyle='--',label="ResNet50 val_loss")
plt.plot(hist_res101["val_loss"],color="orange",linewidth=1.5,linestyle='--',label="ResNet101 val_loss")
plt.plot(hist_ince["val_loss"],color="green",linewidth=1.5,linestyle='--',label="Inception val_loss")
plt.legend()
plt.title("Loss Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss Values")
plt.grid()
plt.savefig("6.png")
plt.show()
'''

In [ ]:
'''
import pickle
import matplotlib.pyplot as plt
with open('../input/densenet121-e30-diff-drop-hist/densenet121_e30_b32_lr0001_adam_05drop_hist', 'rb') as f:
    hist_1 = pickle.load(f)
    
with open('../input/densenet121-e30-diff-drop-hist/densenet_e30_b32_lr0001_adam_drop_hist', 'rb') as f:
    hist_2 = pickle.load(f)

plt.figure(figsize=(10,8))

plt.plot(hist_1["accuracy"],color="blue",linewidth=3.0,label="drop0.5 accuracy")
plt.plot(hist_2["accuracy"],color="red",linewidth=3.0,label="drop0.35 accuracy")


plt.plot(hist_1["val_accuracy"],color="blue",linewidth=1.5,linestyle='--',label="drop0.5 val_accuracy")
plt.plot(hist_2["val_accuracy"],color="red",linewidth=1.5,linestyle='--',label="drop0.35 val_accuracy")

plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Values")
plt.grid()
plt.savefig("7.png")
plt.show()

plt.figure(figsize=(10,8))

plt.plot(hist_1["loss"],color="blue",linewidth=3.0,label="drop0.5 loss")
plt.plot(hist_2["loss"],color="red",linewidth=3.0,label="drop0.35 loss")


plt.plot(hist_1["val_loss"],color="blue",linewidth=1.5,linestyle='--',label="drop0.5 val_loss")
plt.plot(hist_2["val_loss"],color="red",linewidth=1.5,linestyle='--',label="drop0.35 val_loss")

plt.legend()
plt.title("Loss Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss Values")
plt.grid()
plt.savefig("8.png")
plt.show()
'''

In [ ]:
'''
import pickle
import matplotlib.pyplot as plt
with open('../input/densenet121-e30-diff-drop-hist/densenet121_e30_b32_lr0001_adam_05drop_hist', 'rb') as f:
    hist_05 = pickle.load(f)
    
with open('../input/densenet121-e30-diff-drop-hist/densenet_e30_b32_lr0001_adam_drop_hist', 'rb') as f:
    hist_035 = pickle.load(f)


    
plt.figure(figsize=(10,8))

plt.plot(hist_05["accuracy"],color="blue",linewidth=3.0,label="drop0.5 accuracy")
plt.plot(hist_035["accuracy"],color="red",linewidth=3.0,label="drop0.35 accuracy")


plt.plot(hist_05["val_accuracy"],color="blue",linewidth=1.5,linestyle='--',label="drop0.5 val_accuracy")
plt.plot(hist_r035["val_accuracy"],color="red",linewidth=1.5,linestyle='--',label="drop0.35 val_accuracy")

plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Values")
plt.grid()
plt.savefig("7.png")
plt.show()

plt.figure(figsize=(10,8))

plt.plot(hist_05["loss"],color="blue",linewidth=3.0,label="drop0.5 loss")
plt.plot(hist_035["loss"],color="red",linewidth=3.0,label="drop0.35 loss")


plt.plot(hist_05["val_loss"],color="blue",linewidth=1.5,linestyle='--',label="drop0.5 val_loss")
plt.plot(hist_035["val_loss"],color="red",linewidth=1.5,linestyle='--',label="drop0.35 val_loss")

plt.legend()
plt.title("Loss Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss Values")
plt.grid()
plt.savefig("8.png")
plt.show()
'''

In [ ]:
'''
import pickle
import matplotlib.pyplot as plt
with open('../input/densenet-e100-200-hist/densenet_e200_b8_lr0.0001_adam_drop_hist', 'rb') as f:
    hist_densenet_e200 = pickle.load(f)
with open('../input/densenet-e100-200-hist/densenet_e100_b8_lr0.0001_adam_drop_hist', 'rb') as f:
    hist_densenet_e100 = pickle.load(f)
    
plt.figure(figsize=(8,6))
plt.plot(hist_densenet_e200["accuracy"],color="black",linewidth=3.0,label="lr001 accuracy")
plt.plot(hist_densenet_e100["accuracy"],color="blue",linewidth=3.0,label="lr0005 accuracy")
plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy Values")
plt.grid()
plt.show()

plt.figure(figsize=(8,6))
plt.plot(hist_densenet_e200["loss"],color="black",linewidth=3.0,label="lr001 loss")
plt.plot(hist_densenet_e100["loss"],color="blue",linewidth=3.0,label="lr0005 loss")
plt.legend()
plt.title("Loss Plot")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss Values")
plt.grid()
plt.show()
'''

In [ ]:
'''
#加载和测试训练好的模型
model = tf.keras.models.load_model('../input/densenet-e30-b32-lr0001-adam-drop/densenet_e30_b32_lr0001_adam_drop.h5')
#print(model.summary())
datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
pred_generator = datagen.flow_from_directory(
    base + test_dir, 
    target_size = (img_width, img_height),
    class_mode='categorical',
    shuffle = False,
    batch_size=b_size) 
scores = model.evaluate(pred_generator,use_multiprocessing=True,workers=10)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
'''

In [ ]:
'''
from keras.preprocessing import image 

species = list(train_tbl.index)
def predict(dir):
    img=image.load_img(dir,target_size=(224,224))
    img1=image.img_to_array(img)
    img1=img1/255
    img1=np.expand_dims(img1,[0])
    plt.imshow(img)
    predict = model.predict(img1)
    predict=np.argmax(predict,axis=1)
    result = np.squeeze(model.predict(img1))
    print('\n图中鸟类预测结果为:', species[predict[0]])
    print('\n置信度:', max(result))
predict('../input/bird200/test/CALIFORNIA QUAIL/5.jpg')
'''